In [1]:
# package
import numpy as np 
import pandas as pd 
import os
import json
from pandas.io.json import json_normalize
import ast
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import seaborn as sns
%matplotlib notebook
from scipy.stats import skew, boxcox
# import plotly.offline as py
# py.init_notebook_mode(connected=True)
# import plotly.graph_objs as go
# import plotly.tools as tls
from mpl_toolkits.mplot3d import Axes3D
import ast
import re
# import yaml
import json
from collections import Counter
# from nltk.corpus import stopwords
# from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold
# import xgboost as xgb
# import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
# import eli5
import time
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
import warnings  
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [1]:
# method
def date_features(df):
    df[['release_month','release_day','release_year']]=df['release_date'].str.split('/',expand=True).replace(np.nan, -1).astype(int)
    # 연도 끝 두자리수만 있기 때문에 앞에 19/20 붙이기
    df.loc[ (train['release_year'] <= 19) & (df['release_year'] < 100), "release_year"] += 2000
    df.loc[ (train['release_year'] > 19) & (df['release_year'] < 100), "release_year"] += 1900
    return df

def text_to_dict(df):
    for column in dict_columns:
        df[column] = df[column].apply(lambda x: {} if pd.isna(x) else ast.literal_eval(x) )
    return df

In [2]:
submission_path = "..input/sample_submission.csv"
train_path = "..input/train.csv"
test_path = "..input/test.csv"

train = pd.read_csv(train_path)
submission = pd.read_csv(submission_path)
test = pd.read_csv(test_path)

NameError: name 'pd' is not defined

# 2.1 **Data Pre-processing & FE** 

## 2.1.1. **date_features 변환**

In [10]:
train = date_features(train)
test = date_features(test)

___
## 2.1.2. **범주형 변수 변환**

## **Json Format Columns to Dictionary Format**
<br>

- Json 형식의 변수들을 사용하기 위해 dictionary 형식으로 변환하여 분석에 이용.

In [11]:
dict_columns = ['belongs_to_collection', 'genres', 'production_companies',
                'production_countries', 'spoken_languages', 'Keywords', 'cast', 'crew']

train = text_to_dict(train)
test = text_to_dict(test)

## **Belong_to collection**

In [13]:
train['collection_name'] = train['belongs_to_collection'].apply(lambda x: x[0]['name'] if x != {} else 0)
train['has_collection'] = train['belongs_to_collection'].apply(lambda x: len(x) if x != {} else 0)

test['collection_name'] = test['belongs_to_collection'].apply(lambda x: x[0]['name'] if x != {} else 0)
test['has_collection'] = test['belongs_to_collection'].apply(lambda x: len(x) if x != {} else 0)

train = train.drop(['belongs_to_collection'], axis=1)
test = test.drop(['belongs_to_collection'], axis=1)

## **Genres**
<br>

In [14]:
train['num_genres'] = train['genres'].apply(lambda x: len(x) if x != {} else 0)
train['all_genres'] = train['genres'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')

test['num_genres'] = test['genres'].apply(lambda x: len(x) if x != {} else 0)
test['all_genres'] = test['genres'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')

## **production_companies**

In [15]:
train['num_production_companies'] = train['production_companies'].apply(lambda x: len(x) if x != {} else 0)
train['all_production_companies'] = train['production_companies'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')

test['num_production_companies'] = test['production_companies'].apply(lambda x: len(x) if x != {} else 0)
test['all_production_companies'] = test['production_companies'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')

## **production_countries**

In [16]:
train['num_production_countries'] = train['production_countries'].apply(lambda x: len(x) if x != {} else 0)
train['all_production_countries'] = train['production_countries'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')

test['num_production_countries'] = test['production_countries'].apply(lambda x: len(x) if x != {} else 0)
test['all_production_countries'] = test['production_countries'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')

## **Cast**

In [17]:
# list_of_cast_names = list(train['cast'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)
train['num_cast'] = train['cast'].apply(lambda x: len(x) if x != {} else 0)
train['all_cast'] = train['cast'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')

test['num_cast'] = test['cast'].apply(lambda x: len(x) if x != {} else 0)
test['all_cast'] = test['cast'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')

## **Crew**

In [18]:
# list_of_crew_names = list(train['crew'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)
train['num_crew'] = train['crew'].apply(lambda x: len(x) if x != {} else 0)
train['all_crew'] = train['crew'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')

test['num_crew'] = test['crew'].apply(lambda x: len(x) if x != {} else 0)
test['all_crew'] = test['crew'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')

## **spoken_languages**

In [20]:
# list_of_spokenlanguage_names = list(train['spoken_languages'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)
train['num_spoken_languages'] = train['spoken_languages'].apply(lambda x: len(x) if x != {} else 0)
train['all_spoken_languages'] = train['spoken_languages'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')

test['num_spoken_languages'] = test['spoken_languages'].apply(lambda x: len(x) if x != {} else 0)
test['all_spoken_languages'] = test['spoken_languages'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')

KeyError: 'spoken_languages'

## **keyword**

In [21]:
# list_of_Keywords = list(train['Keywords'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)
train['num_Keywords'] = train['Keywords'].apply(lambda x: len(x) if x != {} else 0)
train['all_Keywords'] = train['Keywords'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')
test['num_Keywords'] = test['Keywords'].apply(lambda x: len(x) if x != {} else 0)
test['all_Keywords'] = test['Keywords'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')

## 2.1.3. **Missing data**
<br>

- 결측치가 존재하는 변수들은 NaN값 유무에 따른 이산화형 변수로 변경하여 분석에 이용.

## **homepage**

In [22]:
train['has_homepage'] = 1
train.loc[pd.isnull(train['homepage']) ,"has_homepage"] = 0
test['has_homepage'] = 1
test.loc[pd.isnull(test['homepage']) ,"has_homepage"] = 0

## **tagline**

In [23]:
train['isTaglineNA'] = 0
train.loc[pd.isnull(train['tagline']) ,"isTaglineNA"] = 1
test['isTaglineNA'] = 0
test.loc[pd.isnull(test['tagline']) ,"isTaglineNA"] = 1

## **original, title**

In [24]:
train['isTitleDifferent'] = 1
train.loc[ train['original_title'] == train['title'] ,"isTitleDifferent"] = 0
test['isTitleDifferent'] = 1
test.loc[ test['original_title'] == test['title'] ,"isTitleDifferent"] = 0

## **original_languages**

In [25]:
train['isOriginalLanguageEng'] = 0
test['isOriginalLanguageEng'] = 0

train.loc[train['original_language'] == "en" ,"isOriginalLanguageEng"] = 1
test.loc[test['original_language'] == "en" ,"isOriginalLanguageEng"] = 1

## **spoken_languages**

In [26]:
train['isSpokenLanguageEng'] = 0
train.loc[train['all_spoken_languages'] == "English" ,"isSpokenLanguageEng"] = 1
test['isSpokenLanguageEng'] = 0
test.loc[test['all_spoken_languages'] == "English" ,"isSpokenLanguageEng"] = 1

# **2.1.4. 연속형 변수 변환**
- 연속형 변수들 중에서 치우친 분포를 가진 변수들은 log를 취하여 분석에 이용.

In [27]:
# Putting log variables for skewed data 
train['log_budget']=np.log1p(train['budget'])
test['log_budget']=np.log1p(test['budget'])

train['log_runtime']=np.log1p(train['runtime'])
test['log_runtime']=np.log1p(test['runtime'])

# **3. Data modeling**

In [28]:
# Formating for modeling

used_features = ['release_year', 'num_genres', 'all_genres',
       'num_production_companies', 'all_production_companies',
       'num_production_countries', 'all_production_countries', 'num_cast',
       'all_cast', 'num_crew', 'all_crew', 'num_spoken_languages',
       'all_spoken_languages', 'num_Keywords', 'all_Keywords', 'has_homepage',
       'isTaglineNA', 'isTitleDifferent', 'log_budget', 'log_runtime', 
                 'isOriginalLanguageEng', 'isSpokenLanguageEng']

X = train[used_features]
y = np.log1p(train['revenue'])
 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## **base model**

In [1]:
lgb_model = lgb.LGBMRegressor(**params, n_estimators = 10000, nthread = 4, n_jobs = -1)
lgb_model.fit(X_train, y_train, 
        eval_set=[(X_train, y_train), (X_test, y_test)], eval_metric='rmse',
        verbose=1000, early_stopping_rounds=200)

print("훈련 점수: {:.2f}".format(lgb_model.score(X_train, y_train)))
print("테스트 점수: {:.2f}".format(lgb_model.score(X_test, y_test)))

eli5.show_weights(lgb_model, feature_filter=lambda x: x != '<BIAS>')

y_pred = lgb_model.predict(test[used_features])

NameError: ignored